In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import re
import logging

# Thiết lập logging cơ bản để theo dõi quá trình
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Hàm Tiền xử lý Văn bản ---
def clean_text(text):
    """Làm sạch văn bản: lowercase, xóa ký tự đặc biệt, chuẩn hóa khoảng trắng."""
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'\W+', ' ', text)  # Thay thế non-word characters bằng khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip() # Chuẩn hóa khoảng trắng
    return text

# --- Bước 1: Tải và Tiền xử lý Dữ liệu ---
try:
    # Cần thay thế đường dẫn này bằng đường dẫn thực tế trên máy bạn
    train_df = pd.read_csv(r"C:\Users\HELLO\Downloads\train.csv")
    test_df = pd.read_csv(r"C:\Users\HELLO\Downloads\test.csv")
    logging.info(f"Tải dữ liệu thành công. Mẫu huấn luyện: {len(train_df)}, Mẫu kiểm tra: {len(test_df)}")
except FileNotFoundError:
    logging.error("Lỗi: Không tìm thấy file train.csv hoặc test.csv. Vui lòng kiểm tra đường dẫn.")
    exit()

# Áp dụng tiền xử lý lên cột 'text'
train_df['text'] = train_df['text'].apply(clean_text)
test_df['text'] = test_df['text'].apply(clean_text)

# Chuẩn bị dữ liệu huấn luyện và kiểm tra
X = train_df['text']
y = train_df['label']
X_test = test_df['text']

# Chia train thành train (80%) và validation (20%) để đánh giá
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
logging.info(f"Dữ liệu đã chia: Train={len(X_train)}, Validation={len(X_val)}")

# --- Bước 2: Xây dựng Pipeline (TF-IDF + Multinomial Naive Bayes) ---
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1, 2),        # Sử dụng unigram và bigram
        max_features=25000,        # Giới hạn 25k đặc trưng
        min_df=5                   # Từ phải xuất hiện ít nhất 5 lần
    )),
    ('clf', MultinomialNB(alpha=1.0)) # Thuật toán Naive Bayes
])

# --- Bước 3: Huấn luyện Mô hình ---
logging.info("Bắt đầu huấn luyện mô hình Multinomial Naive Bayes...")
pipeline.fit(X_train, y_train)
logging.info("Huấn luyện mô hình hoàn tất.")

# --- Bước 4: Đánh giá trên Validation Set ---
y_pred_val = pipeline.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred_val)

print("\n" + "="*50)
print(f"HIỆU SUẤT TRÊN TẬP VALIDATION: {val_accuracy:.4f}")
print("="*50)

# KHẮC PHỤC LỖI: Chuyển pipeline.classes_ thành danh sách chuỗi trước khi dùng làm target_names
class_names = [str(c) for c in pipeline.classes_]
print(classification_report(y_val, y_pred_val, target_names=class_names))

print("="*50)

# --- Bước 5: Dự đoán trên Test Set và Tạo Submission ---
logging.info("Bắt đầu dự đoán trên tập kiểm tra và tạo file submission...")
test_predictions = pipeline.predict(X_test)

submission_df = pd.DataFrame({
    'id': test_df['id'],
    'label': test_predictions
})
submission_df.to_csv('submission.csv', index=False)
logging.info("✅ File submission.csv đã được tạo thành công! Sẵn sàng để nộp.")

2025-10-01 09:28:38,777 - INFO - Tải dữ liệu thành công. Mẫu huấn luyện: 9140, Mẫu kiểm tra: 2286
2025-10-01 09:28:38,875 - INFO - Dữ liệu đã chia: Train=7312, Validation=1828
2025-10-01 09:28:38,876 - INFO - Bắt đầu huấn luyện mô hình Multinomial Naive Bayes...
2025-10-01 09:28:39,012 - INFO - Huấn luyện mô hình hoàn tất.
2025-10-01 09:28:39,047 - INFO - Bắt đầu dự đoán trên tập kiểm tra và tạo file submission...
2025-10-01 09:28:39,085 - INFO - ✅ File submission.csv đã được tạo thành công! Sẵn sàng để nộp.



HIỆU SUẤT TRÊN TẬP VALIDATION: 0.8649
              precision    recall  f1-score   support

           0       0.82      0.94      0.87       852
           1       1.00      0.01      0.03        73
           2       0.92      0.86      0.89       903

    accuracy                           0.86      1828
   macro avg       0.91      0.61      0.60      1828
weighted avg       0.88      0.86      0.85      1828

